# Portfolio Optimization

In [4]:
#!pip install cvxpy

In [11]:
import numpy as np
import pandas as pd
import pickle
import cvxpy as cp
import os

In [33]:
predictor = pd.read_csv('../Step3-Portfolio_Construction/predictor.csv', index_col=0)
predictor2 = pd.read_csv('../Step3-Portfolio_Construction/predictor2.csv', index_col=0)

In [34]:
predictor.head()

,AAPL,AAP,AAL,ABC,ABBV,ABI,BVSN,AES,AMGN,AZO,...,VFC,VAR,VRTS,VRTX,VRSN,VRSK,VTR,VZ,VTSS,VSTNQ
1996-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Get Returns
rets = pd.read_hdf('../Step1-Data/1-rets.h5')
rets = rets.loc[:,predictor.columns]
rets.tail()

,AAPL,AAP,AAL,ABC,ABBV,ABI,BVSN,AES,AMGN,AZO,...,VFC,VAR,VRTS,VRTX,VRSN,VRSK,VTR,VZ,VTSS,VSTNQ
2021-08-02,-0.002331,0.003065,-0.015702,0.004584,-0.007309,NaN,NaN,-0.005485,-0.007245,0.006430,...,0.020324,NaN,NaN,-0.016172,0.000324,0.000316,-0.008364,-0.000359,NaN,NaN
2021-08-03,0.012644,0.012646,-0.005483,0.007985,0.006929,NaN,NaN,0.014849,0.017891,0.015973,...,-0.003177,NaN,NaN,0.008723,-0.003096,-0.007947,0.003374,-0.000538,NaN,NaN
2021-08-04,-0.002782,-0.026602,-0.025063,0.014874,-0.010065,NaN,NaN,-0.011706,-0.064610,-0.009162,...,-0.028197,NaN,NaN,-0.002299,-0.008759,-0.005358,-0.012777,-0.006460,NaN,NaN
2021-08-05,0.000749,-0.015882,0.075064,-0.029152,0.000348,NaN,NaN,0.051184,0.024878,-0.003216,...,0.016274,NaN,NaN,0.017035,0.004629,0.012961,0.021458,-0.001264,NaN,NaN
2021-08-06,-0.004767,-0.003005,0.005261,-0.010255,-0.005820,NaN,NaN,-0.010060,-0.016411,-0.005953,...,0.000745,NaN,NaN,-0.011429,-0.004468,-0.009373,-0.028009,-0.001447,NaN,NaN


In [62]:
sample_predictor = predictor['2021-01-01':].dropna(how='all',axis=1)
sample_rets = rets['2021-01-01':].dropna(how='all',axis=1)

Let the weight of our portfolio/assets be $w_i$ We want to optimize our predictors subject to the following constraints:

* Each asset has a weight of less than 3\%: $|w_i| < 0.03 ~\forall~ i$
* The absolute values of our weights need to sum to our leverage: $\sum_i |w_i| = 1$
* The weights need to sum to zero: $\sum_i w_i = 0$

We can put these into the cvxpy package and have it solve for our weights:

In [56]:
n = len(sample_predictor.columns)
w = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
omega = (sample_rets.cov()).values

In [58]:
sample_rets.cov()

,AAPL,AAP,AAL,ABC,ABBV,AES,AMGN,AZO,CAG,BMY,...,VMC,VIAC,VLO,VNO,VFC,VRTX,VRSN,VRSK,VTR,VZ
AAPL,0.000279,-0.000020,0.000003,0.000018,0.000054,0.000059,0.000058,0.000019,-1.373797e-05,0.000014,...,-0.000005,-0.000007,-0.000012,-0.000036,0.000070,0.000048,0.000099,0.000086,0.000045,-0.000009
AAP,-0.000020,0.000293,0.000087,0.000129,0.000056,0.000104,0.000048,0.000187,4.645432e-05,0.000047,...,0.000127,0.000052,0.000117,0.000086,0.000170,0.000022,-0.000005,0.000068,0.000092,0.000069
AAL,0.000003,0.000087,0.000946,0.000026,0.000051,0.000201,0.000017,0.000004,3.249936e-05,0.000013,...,0.000129,0.000353,0.000363,0.000379,0.000183,0.000030,-0.000047,-0.000051,0.000177,-0.000006
ABC,0.000018,0.000129,0.000026,0.000278,0.000028,0.000048,0.000056,0.000104,2.422474e-05,0.000036,...,0.000094,0.000074,0.000033,0.000076,0.000114,0.000046,-0.000022,0.000079,0.000080,0.000048
ABBV,0.000054,0.000056,0.000051,0.000028,0.000153,0.000089,0.000048,0.000053,2.783476e-06,0.000073,...,0.000027,-0.000025,0.000057,0.000014,0.000046,0.000064,0.000029,0.000016,0.000038,0.000032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VRTX,0.000048,0.000022,0.000030,0.000046,0.000064,0.000031,0.000092,0.000025,3.421274e-07,0.000045,...,0.000016,-0.000017,0.000009,0.000036,0.000010,0.000284,0.000021,0.000037,0.000035,0.000030
VRSN,0.000099,-0.000005,-0.000047,-0.000022,0.000029,0.000031,0.000041,0.000015,2.774514e-06,0.000027,...,-0.000016,-0.000140,-0.000039,-0.000034,0.000011,0.000021,0.000169,0.000067,0.000030,-0.000002
VRSK,0.000086,0.000068,-0.000051,0.000079,0.000016,0.000052,0.000048,0.000073,1.377889e-05,0.000002,...,-0.000006,-0.000065,-0.000020,-0.000027,0.000046,0.000037,0.000067,0.000243,0.000071,0.000028
VTR,0.000045,0.000092,0.000177,0.000080,0.000038,0.000125,0.000035,0.000061,3.938256e-05,0.000019,...,0.000071,0.000063,0.000106,0.000174,0.000097,0.000035,0.000030,0.000071,0.000255,0.000036


In [59]:
risk = cp.quad_form(w, omega)
prob = cp.Problem(cp.Maximize(w*predictor - gamma*risk), 
               [cp.sum(w) == 1, 
                w >= 0])

Exception: Invalid dimensions for arguments.

In [61]:
w

Variable((795,))

In [60]:
np.shape(omega)

(429, 429)